In [28]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [29]:
df = pd.read_csv('original_scored_eventlog.csv')
df['time'] = df['time'].astype(float)

# 연속중복 행 제거
city→concession→individual→trip_n→trip_n→trip_4→cancel 같은 경우 

city→concession→individual→trip_n→trip_4→cancel로 만들어야 함

In [30]:
df['reps'] = 0

for i in tqdm(range(len(df)-1)):
        if (df['ID'][i] == df['ID'][i+1]) and (df['instance'][i] == df['instance'][i+1]):
            df['reps'][i+1] = 1

100%|██████████| 322873/322873 [00:05<00:00, 53841.55it/s]


In [31]:
# 연속으로 반복되는 행동이 아닌 경우 (reps=0)인 경우만 남김
df = df[(df['reps'] != 1)]
df = df.reset_index(drop = True)
df

,Unnamed: 0,ID,cnt,schoolid,StIDStd,event,time,event_number,event_value,network,fare_type,ticket_type,number_trips,OECD,credit,timestamp,instance,instance_id,reps
0,1,ARE000018904852,ARE,189,4852,START_ITEM,1186.7,1,Start,NaN,NaN,NaN,NaN,0,0,2021-12-22 14:19:46,1,ARE000018904852-01,0
1,2,ARE000018904852,ARE,189,4852,ACER_EVENT,1195.6,2,country_trains,country_trains,NaN,NaN,0.0,0,0,2021-12-22 14:19:55,3,ARE000018904852-03,0
2,3,ARE000018904852,ARE,189,4852,ACER_EVENT,1202.8,3,full_fare,country_trains,full_fare,NaN,0.0,0,0,2021-12-22 14:20:02,4,ARE000018904852-04,0
3,4,ARE000018904852,ARE,189,4852,ACER_EVENT,1217.2,4,daily,country_trains,full_fare,daily,0.0,0,0,2021-12-22 14:20:17,7,ARE000018904852-07,0
4,5,ARE000018904852,ARE,189,4852,ACER_EVENT,1223.7,5,Buy,country_trains,full_fare,daily,0.0,0,0,2021-12-22 14:20:23,10,ARE000018904852-10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313263,322870,USA000007902365,USA,79,2365,ACER_EVENT,111.5,16,city_subway,city_subway,NaN,NaN,0.0,1,0,2021-12-22 14:01:51,2,USA000007902365-02,0
313264,322871,USA000007902365,USA,79,2365,ACER_EVENT,112.0,17,concession,city_subway,concession,NaN,0.0,1,0,2021-12-22 14:01:52,5,USA000007902365-05,0
313265,322872,USA000007902365,USA,79,2365,ACER_EVENT,113.1,18,daily,city_subway,concession,daily,0.0,1,0,2021-12-22 14:01:53,7,USA000007902365-07,0
313266,322873,USA000007902365,USA,79,2365,ACER_EVENT,128.6,19,Buy,city_subway,concession,daily,0.0,1,0,2021-12-22 14:02:08,10,USA000007902365-10,0


In [32]:
stu_id = df['ID'].unique().tolist()
len(stu_id)

30453

In [64]:
df.to_csv('data_log.csv', index=False)

# 시퀀스 변수

In [33]:
df_action = df[['ID','time','event_value', 'event_number','instance','credit']]

In [34]:
df_action["sequence"] = np.nan

In [35]:
df_action

,ID,time,event_value,event_number,instance,credit,sequence
0,ARE000018904852,1186.7,Start,1,1,0,NaN
1,ARE000018904852,1195.6,country_trains,2,3,0,NaN
2,ARE000018904852,1202.8,full_fare,3,4,0,NaN
3,ARE000018904852,1217.2,daily,4,7,0,NaN
4,ARE000018904852,1223.7,Buy,5,10,0,NaN
...,...,...,...,...,...,...,...
313263,USA000007902365,111.5,city_subway,16,2,0,NaN
313264,USA000007902365,112.0,concession,17,5,0,NaN
313265,USA000007902365,113.1,daily,18,7,0,NaN
313266,USA000007902365,128.6,Buy,19,10,0,NaN


- city_subway 2
- concession 5
- daily 7
- cancel 11
- individual 6
- trip_n 8
- trip_4 9
- Buy 10


- a1 city→concession→daily→cancel                      2-5-7-11
- b1 city→concession→individual→trip_n→trip_4→cancel 2-5-6-8-9-11
- b2 city→concession→individual→trip_n→trip_4→Buy    2-5-6-8-9-10
- b3 city→concession→individual→trip_4→cancel        2-5-6-9-11
- b4 city→concession→individual→trip_4→Buy           2-5-6-9-10

In [36]:
for i in tqdm(range(len(df_action))):
    if(df_action['instance'][i]==2) and (df_action['instance'][i+1]==5) and (df_action['instance'][i+2]==7) and (df_action['instance'][i+3]==11):
        df_action['sequence'][i:i+3]= ''
        df_action['sequence'][i+3]= 'a1'
        
    if(df_action['instance'][i]==2) and (df_action['instance'][i+1]==5) and (df_action['instance'][i+2]==6) and (df_action['instance'][i+3]==8) and (df_action['instance'][i+4]==9) and (df_action['instance'][i+5]==11):
        df_action['sequence'][i:i+5]= ''
        df_action['sequence'][i+5] = 'b1'
        
    if(df_action['instance'][i]==2) and (df_action['instance'][i+1]==5) and (df_action['instance'][i+2]==6) and (df_action['instance'][i+3]==8) and (df_action['instance'][i+4]==9) and (df_action['instance'][i+5]==10):
        df_action['sequence'][i:i+5]= ''
        df_action['sequence'][i+5] = 'b2'
        
    if(df_action['instance'][i]==2) and (df_action['instance'][i+1]==5) and (df_action['instance'][i+2]==6) and (df_action['instance'][i+3]==9) and (df_action['instance'][i+4]==11):
        df_action['sequence'][i:i+4]= ''
        df_action['sequence'][i+4]= 'b3'
        
    if(df_action['instance'][i]==2) and (df_action['instance'][i+1]==5) and (df_action['instance'][i+2]==6) and (df_action['instance'][i+3]==9) and (df_action['instance'][i+4]==10):
        df_action['sequence'][i:i+4]= ''
        df_action['sequence'][i+4]= 'b4'

100%|██████████| 313268/313268 [01:32<00:00, 3399.63it/s]


In [47]:
df_action.fillna('c', inplace=True)
df_action.tail(50)

,ID,time,event_value,event_number,instance,credit,sequence
313218,USA000007902371,1114.3,Buy,13,10,1,c
313219,USA000007902371,1126.8,End,14,12,1,c
313220,USA000007902381,1387.3,Start,1,1,1,c
313221,USA000007902381,1398.6,city_subway,2,2,1,c
313222,USA000007902381,1399.6,concession,3,5,1,c
313223,USA000007902381,1418.6,individual,4,6,1,c
313224,USA000007902381,1427.2,trip_n,5,8,1,c
313225,USA000007902381,1432.7,trip_4,13,9,1,c
313226,USA000007902381,1436.8,trip_n,14,8,1,c
313227,USA000007902381,1438.0,trip_4,15,9,1,c


In [48]:
# start, end 삭제
df_action_new = df_action[(df_action['instance'] != 1)]
df_action_new = df_action_new[(df_action_new['instance'] != 12)]
df_action_new = df_action_new.reset_index(drop = True)
df_action_new.tail(50)

,ID,time,event_value,event_number,instance,credit,sequence
252274,USA000007902371,1100.9,individual,8,6,1,c
252275,USA000007902371,1103.3,trip_n,9,8,1,c
252276,USA000007902371,1106.8,trip_4,10,9,1,c
252277,USA000007902371,1110.1,trip_n,11,8,1,c
252278,USA000007902371,1110.6,trip_4,12,9,1,c
252279,USA000007902371,1114.3,Buy,13,10,1,c
252280,USA000007902381,1398.6,city_subway,2,2,1,c
252281,USA000007902381,1399.6,concession,3,5,1,c
252282,USA000007902381,1418.6,individual,4,6,1,c
252283,USA000007902381,1427.2,trip_n,5,8,1,c


In [49]:
new_stu_id = df_action_new['ID'].unique().tolist()
len(new_stu_id)

29376

In [51]:
# a, b는 마지막 한칸씩만 남기기
df_action_new = df_action_new[(df_action_new['sequence'] != '')]
df_action_new.reset_index(drop = True, inplace=True)
df_action_new

,ID,time,event_value,event_number,instance,credit,sequence
0,ARE000018904852,1195.6,country_trains,2,3,0,c
1,ARE000018904852,1202.8,full_fare,3,4,0,c
2,ARE000018904852,1217.2,daily,4,7,0,c
3,ARE000018904852,1223.7,Buy,5,10,0,c
4,ARE000018904861,1004.7,country_trains,2,3,0,c
...,...,...,...,...,...,...,...
147838,USA000007902365,110.8,Cancel,15,11,0,c
147839,USA000007902365,111.5,city_subway,16,2,0,c
147840,USA000007902365,112.0,concession,17,5,0,c
147841,USA000007902365,113.1,daily,18,7,0,c


In [52]:
df_action_new['reps'] = 0

for i in tqdm(range(len(df_action_new)-1)):
    if (df_action_new['ID'][i] == df_action_new['ID'][i+1]) and (df_action_new['sequence'][i] == 'c') and (df_action_new['sequence'][i+1]=='c'):
        df_action_new['reps'][i] = 1

100%|██████████| 147842/147842 [00:08<00:00, 16844.44it/s]


In [53]:
df_action_new.tail(50)

,ID,time,event_value,event_number,instance,credit,sequence,reps
147793,USA000015004612,110.8,Buy,10,10,1,b4,0
147794,USA000015004637,113.1,Cancel,5,11,1,a1,0
147795,USA000015004637,114.2,country_trains,6,3,1,c,1
147796,USA000015004637,116.9,Cancel,7,11,1,c,0
147797,USA000015004637,129.0,Buy,12,10,1,b4,0
147798,USA000007902358,143.7,Cancel,5,11,1,a1,0
147799,USA000007902358,158.0,Buy,10,10,1,b4,0
147800,USA000007902371,1097.6,Cancel,5,11,1,a1,0
147801,USA000007902371,1098.5,city_subway,6,2,1,c,1
147802,USA000007902371,1099.2,concession,7,5,1,c,1


In [54]:
# 'c'가 연속으로 중복된 경우 삭제
df_action_new = df_action_new[(df_action_new['reps'] == 0)]
df_action_new = df_action_new.reset_index(drop = True)
df_action_new.tail(50)

,ID,time,event_value,event_number,instance,credit,sequence,reps
51167,USA000005901731,313.7,Cancel,17,11,0,a1,0
51168,USA000005901731,328.5,Buy,27,10,0,c,0
51169,USA000010603237,110.1,Buy,5,10,0,c,0
51170,USA000010603226,880.3,Cancel,5,11,1,a1,0
51171,USA000010603226,898.1,Buy,10,10,1,b4,0
51172,USA000010603244,144.9,Buy,7,10,0,b4,0
51173,USA000013003978,1160.5,Buy,6,10,0,b4,0
51174,USA000013003997,141.7,Buy,7,10,0,c,0
51175,USA000013003967,813.7,Buy,6,10,0,b4,0
51176,USA000008902703,966.9,Cancel,5,11,0,a1,0


In [55]:
# 문자열 합치기
full_sequence = []
for i in tqdm(new_stu_id):
    a = df_action_new[df_action_new['ID']==i]
    
    seq1 = a['sequence'].tolist()
    seq2 = ''.join(seq1)
    full_sequence.append(seq2)

100%|██████████| 29376/29376 [01:29<00:00, 327.03it/s]


In [56]:
# 학생 당 점수 붙이기
credit = []
for i in tqdm(new_stu_id):
    a = df_action_new[df_action_new['ID']==i]
    stu_credit = min(a['credit'])
    credit.append(stu_credit)

100%|██████████| 29376/29376 [01:34<00:00, 310.48it/s]


In [57]:
df_seq = pd.DataFrame({'stu_id':new_stu_id, 'credit':credit, 'sequence':full_sequence})
df_seq.head(20)

,stu_id,credit,sequence
0,ARE000018904852,0,c
1,ARE000018904861,0,c
2,ARE000018904843,0,c
3,ARE000018904832,0,c
4,ARE000040210135,0,c
5,ARE000040210143,0,c
6,ARE000040210127,0,c
7,ARE000004801214,0,c
8,ARE000026106641,0,c
9,ARE000026106640,0,c


In [58]:
# b1,b3 합치고, b2, b4 합치기
df_seq['sequence'] = df_seq['sequence'].str.replace('b3', 'b1')
df_seq['sequence'] = df_seq['sequence'].str.replace('b4', 'b2')
df_seq.head(20)

,stu_id,credit,sequence
0,ARE000018904852,0,c
1,ARE000018904861,0,c
2,ARE000018904843,0,c
3,ARE000018904832,0,c
4,ARE000040210135,0,c
5,ARE000040210143,0,c
6,ARE000040210127,0,c
7,ARE000004801214,0,c
8,ARE000026106641,0,c
9,ARE000026106640,0,c


In [59]:
# 시퀀스 종류
seq = df_seq['sequence'].tolist()
seq = list(set(seq))
seq.sort(key=len)
seq

['c',
 'a1',
 'b2',
 'b1',
 'b1c',
 'a1c',
 'cb1',
 'ca1',
 'cb2',
 'b2c',
 'b1b2',
 'a1b2',
 'cb1c',
 'ca1c',
 'a1b1',
 'b1cb2',
 'b1cb1',
 'a1b1c',
 'b1a1c',
 'cb1b2',
 'a1a1c',
 'a1cb2',
 'ca1b2',
 'b1b1c',
 'ca1b1c',
 'ca1cb2',
 'a1cb1c',
 'b1ca1c',
 'a1a1b2',
 'b1b1b2',
 'cb1cb2',
 'cb1b1c',
 'b1cb1c',
 'cb1a1c',
 'a1ca1c',
 'ca1ca1',
 'a1b1b2',
 'b1a1b2',
 'ca1a1c',
 'a1ca1b2',
 'a1b1cb2',
 'b1b1cb2',
 'b1b1a1c',
 'b1a1cb2',
 'ca1b1b2',
 'a1b1a1c',
 'b1cb1b2',
 'b1a1b2c',
 'cb1cb1c',
 'cb1a1b2',
 'a1cb1b2',
 'ca1a1b2',
 'ca1ca1c',
 'a1a1a1c',
 'cb1ca1c',
 'a1a1b1c',
 'a1a1cb2',
 'b1ca1b2',
 'b1a1b1c',
 'ca1cb1c',
 'a1b1a1b2',
 'a1b1ca1c',
 'b1ca1b1c',
 'a1ca1a1c',
 'b1a1ca1c',
 'ca1cb1b2',
 'a1cb1cb2',
 'b1a1b1b2',
 'cb1b1cb2',
 'a1a1ca1c',
 'b1cb1cb2',
 'ca1a1cb2',
 'a1a1a1b2',
 'b1a1a1b2',
 'ca1a1b1c',
 'b1b1b1b2',
 'a1b1cb1c',
 'cb1ca1b2',
 'b1b1a1b2',
 'a1ca1cb2',
 'ca1a1a1c',
 'cb1a1cb2',
 'a1cb1a1c',
 'cb1a1a1c',
 'ca1ca1b2',
 'ca1b1cb2',
 'b1ca1cb2',
 'a1ca1b1c',
 'a1a1b1b

In [60]:
len(seq)

377

In [63]:
df_seq.to_csv('data_seq.csv', index=False)